In [1]:
import warnings
warnings.simplefilter('ignore')
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import pandas as pd
import numpy as np
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
import torch.nn.functional as F
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import TensorDataset, DataLoader
!pip install pyarrow

In [2]:
# 读取数据
train_data = pd.read_feather('./input/train.feather')
labels = pd.read_csv('/openbayes/input/input0/train_labels.csv')
train_data = pd.merge(train_data, labels, on='customer_ID', how='left')  # 合并标签列到训练数据集中
test_data = pd.read_feather('./input/test.feather')

# 标签
label_name = 'target'

# 标准化数值特征
numerical_cols = [col for col in train_data.columns if col not in ['customer_ID', 'S_2', label_name] and train_data[col].dtype != 'O']

# 使用均值填充数值特征的缺失值
train_data[numerical_cols] = train_data[numerical_cols].fillna(train_data[numerical_cols].mean())
test_data[numerical_cols] = test_data[numerical_cols].fillna(train_data[numerical_cols].mean())

scaler = StandardScaler()
train_data[numerical_cols] = scaler.fit_transform(train_data[numerical_cols])
test_data[numerical_cols] = scaler.transform(test_data[numerical_cols])

# 类别特征处理 - 使用 Embedding 层替代 DNN 处理
cat_features = ["B_30", "B_38", "D_114", "D_116", "D_117", "D_120", "D_126", "D_63", "D_64", "D_66", "D_68"]

class CategoricalEmbedding(nn.Module):
    def __init__(self, input_dim, embedding_dim):
        super(CategoricalEmbedding, self).__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx=-1)

    def forward(self, x):
        return self.embedding(x)

# 处理缺失值和类别特征编码
for col in cat_features:
    # 使用-1填充缺失值
    train_data[col] = train_data[col].astype('category').cat.codes.replace(np.nan,-1)
    test_data[col] = test_data[col].astype('category').cat.codes.replace(np.nan,-1)

    # 使用训练集中的编码
    unique_values_train = set(train_data[col].dropna().unique())
    unique_values_test = set(test_data[col].dropna().unique())

    # 新增的类别值
    new_values = unique_values_test - unique_values_train

    # 如果测试集中有新的类别值，将它们映射为一个特殊的编码
    if new_values:
        mapping = {val: len(unique_values_train) for val in new_values}
        test_data[col] = test_data[col].apply(lambda x: mapping.get(x, x))

    embedding_dim = min(50, len(unique_values_train) + 1)  # +1 for the padding index
    embedding_layer = CategoricalEmbedding(len(unique_values_train) + 1, embedding_dim)

    # Apply the embedding to the input data
    embedded_values = embedding_layer(torch.LongTensor(train_data[col].values))
    # Concatenate the embedded values to the dataframe
    train_data = pd.concat([train_data, pd.DataFrame(embedded_values.detach().numpy(), columns=[f'{col}_{i}' for i in range(embedding_dim)])], axis=1)

    # Transform on test data
    embedded_test_values = embedding_layer(torch.LongTensor(test_data[col].values))
    test_data = pd.concat([test_data, pd.DataFrame(embedded_test_values.detach().numpy(), columns=[f'{col}_{i}' for i in range(embedding_dim)])], axis=1)

# 删除原始的类别特征列
train_data = train_data.drop(cat_features, axis=1)
test_data = test_data.drop(cat_features, axis=1)

# 时间特征处理
time_features = ["S_2"]
for col in time_features:
    train_data[col] = pd.to_datetime(train_data[col])
    test_data[col] = pd.to_datetime(test_data[col])
    

train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=None)


# 构建 RNN 数据
def prepare_rnn_data(data):
    data['customer_ID'] = data['customer_ID']
    data[time_features] = data[time_features].apply(lambda x: x.astype(np.int64) // 10**9)

    if label_name not in data.columns:
        data[label_name] = 0

    # 取平均值，大于0.5定为1，反之定为0
    data[label_name] = (data.groupby('customer_ID')[label_name].transform('mean') > 0.5).astype(np.float32)

    rnn_data = data.groupby('customer_ID').apply(lambda x: x[time_features + [label_name]].values.tolist()).tolist()
    rnn_data = [torch.tensor(seq, dtype=torch.float32) for seq in rnn_data]

    rnn_data_padded = pad_sequence(rnn_data, batch_first=True, padding_value=0)

    rnn_labels = data.groupby('customer_ID')[label_name].mean().values
    rnn_labels = (rnn_labels > 0.5).astype(np.float32)
    return rnn_data_padded, rnn_labels

train_rnn_data, train_rnn_labels = prepare_rnn_data(train_data)
val_rnn_data, val_rnn_labels = prepare_rnn_data(val_data)
test_rnn_data, _ = prepare_rnn_data(test_data)

# 构建 DNN 数据
def prepare_dnn_data(data):
    data[time_features] = data[time_features].apply(lambda x: x.astype(np.int64) // 10**9)
    
    dnn_data = data.drop(time_features + [label_name], axis=1)
    dnn_data = dnn_data.groupby('customer_ID').mean().reset_index(drop=False)
    dnn_data = dnn_data.drop_duplicates('customer_ID')
    dnn_data = dnn_data.drop(['customer_ID'], axis=1)
    dnn_labels = data.groupby('customer_ID')[label_name].mean().values
    dnn_labels = (dnn_labels > 0.5).astype(np.float32).ravel()
    return dnn_data.values.astype(np.float32), dnn_labels

train_dnn_data, train_dnn_labels = prepare_dnn_data(train_data)
val_dnn_data, val_dnn_labels = prepare_dnn_data(val_data)
#train_dnn_loader = DataLoader(list(zip(train_dnn_data, train_dnn_labels)), batch_size=64, shuffle=True)
#val_dnn_loader = DataLoader(list(zip(val_dnn_data, val_dnn_labels)), batch_size=64, shuffle=True)
test_dnn_data, _ = prepare_dnn_data(test_data)



In [3]:
# 定义新的神经网络模型
class EnhancedNN(nn.Module):
    def __init__(self, rnn_input_size, rnn_hidden_size, dnn_input_size, dnn_hidden_size, output_size, weight_decay=0.0):
        super(EnhancedNN, self).__init__()
        
        self.rnn = nn.LSTM(input_size=rnn_input_size, hidden_size=rnn_hidden_size, batch_first=True)
        self.fc1 = nn.Linear(dnn_input_size, dnn_hidden_size)
        self.fc2 = nn.Linear(rnn_hidden_size + dnn_hidden_size, output_size)

        self.weight_decay = weight_decay

    def forward(self, rnn_input, dnn_input):
        if self.training:
            _, (rnn_out, _) = self.rnn(rnn_input)
        else:
            with torch.no_grad():
                _, (rnn_out, _) = self.rnn(rnn_input)
        # 使用整个序列的信息，而不仅仅是最后一个时间步
        rnn_out = rnn_out.squeeze(dim=0)
        rnn_out = rnn_out.unsqueeze(1)  # 添加一个维度

        dnn_out = self.fc1(dnn_input)
        dnn_out = F.relu(dnn_out)
        #print("rnn_out shape:", rnn_out.shape)
        #print("dnn_out shape:", dnn_out.shape)

        combined_out = torch.cat([rnn_out, dnn_out], dim=-1)
        
    
        if self.weight_decay > 0.0:
            l2_reg = torch.tensor(0.0, requires_grad=True)
            for param in self.parameters():
                l2_reg += torch.norm(param)
            combined_out += self.weight_decay * l2_reg

        output = self.fc2(combined_out)
        output = torch.sigmoid(output)

        return output



# 训练新模型
def train_enhanced_model(model, train_loader, val_loader, criterion, optimizer, epochs=5, patience=3):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Training Enhanced Model...")
    model.to(device)
    model.train()

    best_val_loss = float('inf')
    counter = 0  # 计数连续不改善的周期

    for epoch in range(epochs):
        total_loss = 0.0
        for batch_idx, (rnn_input, dnn_input, labels) in enumerate(tqdm(train_loader, desc=f'Epoch {epoch + 1}/{epochs}')):
            #rnn_input, dnn_input, labels = torch.tensor(rnn_input, dtype=torch.float32), torch.tensor(dnn_input, dtype=torch.float32), torch.tensor(labels, dtype=torch.float32)
            rnn_input, dnn_input, labels = rnn_input.to(device), dnn_input.to(device), labels.to(device)
            #print("rnn_input shape:", rnn_input.shape)
            #print("dnn_input shape:", dnn_input.shape)
            optimizer.zero_grad()
            outputs = model(rnn_input, dnn_input.unsqueeze(1))
            loss = criterion(outputs.squeeze(), labels.squeeze())
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f'Epoch {epoch + 1}/{epochs}, Avg Loss: {avg_loss:.4f}')

        # 在验证集上评估性能
        val_loss, val_accuracy = evaluate_model(model, val_loader, criterion, device)

        print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2%}')

        # 提前停止逻辑
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print(f'Early stopping at epoch {epoch + 1} due to no improvement in validation loss.')
                break

    print('Training finished.')

def evaluate_model(model, val_loader, criterion, device):
    model.eval()
    total_val_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for rnn_input, dnn_input, labels in tqdm(val_loader, desc='Validation'):
            # 将NumPy数组转换为PyTorch张量
            #rnn_input, dnn_input, labels = torch.tensor(rnn_input, dtype=torch.float32), torch.tensor(dnn_input, dtype=torch.float32), torch.tensor(labels, dtype=torch.float32)
            
            #print("rnn_input shape:", rnn_input.shape)
            #print("dnn_input shape:", dnn_input.shape)
            #print("labels shape:", labels.shape)
            # 将输入数据移到设备上
            rnn_input, dnn_input, labels = rnn_input.to(device), dnn_input.to(device), labels.to(device)
            outputs = model(rnn_input, dnn_input.unsqueeze(1))
            loss = criterion(outputs.squeeze(), labels.squeeze())
            total_val_loss += loss.item()
            # 计算正确的预测
            predicted_labels = (outputs > 0.5).float()
            predicted_labels = predicted_labels.view_as(labels)
            #correct_predictions += (predicted_labels == labels).sum().item()
            #total_samples += labels.numel()
            #print(f'Outputs Shape: {outputs.shape}')
            #print(f'Predicted Labels Shape: {predicted_labels.shape}')
            #print(f'Labels Shape: {labels.shape}')
            batch_correct_predictions = (predicted_labels == labels).sum().item()
            batch_total_samples = labels.size(0)  # 使用 size(0) 以确保仅计算批次大小

            correct_predictions += batch_correct_predictions
            total_samples += batch_total_samples
            #print(f'Batch Correct Predictions: {batch_correct_predictions}')
            #print(f'Batch Total Samples: {batch_total_samples}')
            #print(f'Accumulated Correct Predictions: {correct_predictions}')
            #print(f'Accumulated Total Samples: {total_samples}')

    avg_val_loss = total_val_loss / len(val_loader)
    val_accuracy = correct_predictions / total_samples if total_samples > 0 else 0.0
    #print(f'Final Validation Accuracy: {val_accuracy * 100:.2f}%')

    return avg_val_loss, val_accuracy

# 测试 Enhanced 模型
def test_enhanced_model(model, rnn_test_data, dnn_test_data):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Testing Enhanced Model...")
    model.to(device)
    model.eval()
    print("rnn_test_data shape:", rnn_test_data.shape)
    print("dnn_test_data shape:", dnn_test_data.shape)

    # 将测试数据封装进 TensorDataset
    test_dataset = TensorDataset(torch.tensor(rnn_test_data, dtype=torch.float32), 
                                 torch.tensor(dnn_test_data, dtype=torch.float32))
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    enhanced_preds = []
    with torch.no_grad():
        for rnn_input, dnn_input in tqdm(test_loader, desc='Enhanced Testing'):
            rnn_input, dnn_input = rnn_input.to(device), dnn_input.to(device)
            outputs = model(rnn_input, dnn_input.unsqueeze(1))
            enhanced_preds.extend(outputs.cpu().numpy())

    print("Testing over.")
    return np.array(enhanced_preds).flatten()  # 根据需要调整输出格式

In [4]:
# 交叉验证部分
# 使用 StratifiedKFold 进行 K 折交叉验证
num_folds = 5
stratified_kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

# 初始化模型、优化器和损失函数
enhanced_model = EnhancedNN(
    rnn_input_size=train_rnn_data.shape[2],
    rnn_hidden_size=64,
    dnn_input_size=train_dnn_data.shape[1],
    dnn_hidden_size=64,
    output_size=1
)
enhanced_criterion = nn.BCEWithLogitsLoss()
enhanced_optimizer = optim.Adam(enhanced_model.parameters(), lr=0.001)

# 训练和验证模型
for fold, (train_indices, val_indices) in enumerate(stratified_kfold.split(train_rnn_data, train_dnn_labels)):
    print(f'Fold {fold + 1}/{num_folds}')

    # 划分数据
    fold_train_rnn_data, fold_train_dnn_data, fold_train_dnn_labels = train_rnn_data[train_indices], train_dnn_data[train_indices], train_dnn_labels[train_indices]
    fold_val_rnn_data, fold_val_dnn_data, fold_val_dnn_labels = train_rnn_data[val_indices], train_dnn_data[val_indices], train_dnn_labels[val_indices]
    
    # 创建 DataLoader
    fold_train_loader = DataLoader(list(zip(fold_train_rnn_data, fold_train_dnn_data, fold_train_dnn_labels)), batch_size=64, shuffle=True)
    fold_val_loader = DataLoader(list(zip(fold_val_rnn_data, fold_val_dnn_data, fold_val_dnn_labels)), batch_size=64, shuffle=False)

    # 训练 Enhanced 模型，传递验证集数据和提前停止的耐心参数
    train_enhanced_model(enhanced_model, fold_train_loader, fold_val_loader, enhanced_criterion, enhanced_optimizer, epochs=5, patience=3)


Fold 1/5
Training Enhanced Model...


Epoch 1/5: 100%|██████████| 5721/5721 [00:12<00:00, 451.56it/s]


Epoch 1/5, Avg Loss: 0.6501


Validation: 100%|██████████| 1431/1431 [00:01<00:00, 1249.81it/s]


Validation Loss: 0.6486, Validation Accuracy: 88.12%


...

Validation Loss: 0.6424, Validation Accuracy: 89.52%


Epoch 4/5: 100%|██████████| 5721/5721 [00:07<00:00, 739.86it/s]


Epoch 4/5, Avg Loss: 0.6414


Validation: 100%|██████████| 1431/1431 [00:01<00:00, 1255.61it/s]

Validation Loss: 0.6429, Validation Accuracy: 89.08%
Early stopping at epoch 4 due to no improvement in validation loss.
Training finished.


In [5]:
# 在测试集上评估模型，这里需要你根据具体情况实现 test_enhanced_model 函数
test_enhanced_preds = test_enhanced_model(enhanced_model, test_rnn_data, test_dnn_data)

# 输出最终结果
#final_result = pd.DataFrame({'customer_ID': test_data['customer_ID'].values[:len(test_enhanced_preds)], 'prob': test_enhanced_preds.squeeze()})
final_result = pd.DataFrame({'customer_ID': test_data.drop_duplicates('customer_ID')['customer_ID'].values, 'prob': test_enhanced_preds.squeeze()})
# 聚合测试集中相同 customer_ID 的预测结果
#aggregated_test_preds = final_result.groupby('customer_ID')['prob'].mean().reset_index()
# 删除重复的 customer_ID
#final_result_unique = aggregated_test_preds.drop_duplicates('customer_ID')
# 输出最终结果
#final_result_aggregated = pd.DataFrame({'customer_ID': final_result_unique['customer_ID'].values, 'prob': final_result_unique['prob']})

#final_result_aggregated.to_csv('./output/final_result_aggregated.csv', index=False)
final_result.to_csv('./output/final_result.csv', index=False)


Testing Enhanced Model...
rnn_test_data shape: torch.Size([924621, 13, 2])
dnn_test_data shape: (924621, 243)


Enhanced Testing: 100%|██████████| 14448/14448 [00:11<00:00, 1304.09it/s]


Testing over.


In [6]:
dfdfdf = pd.read_csv('./output/final_result.csv')
dfdfdf

,customer_ID,prob
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,1.042290e-37
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,1.294072e-36
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,2.729212e-23
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,2.867651e-01
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,8.769653e-01
...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.000000e+00
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,9.807990e-01
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,2.852487e-02
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,1.845468e-09
